#구글 드라이브 mount

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#필요한 라이브러리 import

In [2]:
import os
import glob
import re
import pickle
import sklearn
import librosa
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras import activations
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam


In [3]:
# F1 score 계산
def get_model_metrics(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    precision = round(precision_score(y_test, pred, average='weighted'), ndigits=3)
    recall = round(recall_score(y_test, pred, average='weighted'), ndigits=3)
    model_f1_score = round(f1_score(y_test, pred, average='weighted'), ndigits=3)
    print('Confusion Matrix')
    print(confusion)
    print('==='*10)
    print('정밀도:{}, 재현율:{}, f1_score:{}'.format(precision, recall, model_f1_score))

##Text 모델

데이터셋 불러오기

In [4]:
file_path="/content/gdrive/MyDrive/Human_understand/KEMDy19/merged_bio_text.csv"


In [5]:
merged_txt_dataset = pd.read_csv(file_path, encoding='utf-8-sig', usecols=['Seg', 'tokenized', 'Label_angry', 'Label_disgust', 'Label_fear', 'Label_happy', 'Label_neutral', 'Label_sad', 'Label_surprise'])

데이터셋을 x_data, y_data로 분리

In [6]:
X_data_txt= merged_txt_dataset['tokenized'].astype(str)
Y_data_txt= merged_txt_dataset[['Label_angry', 'Label_disgust', 'Label_fear', 'Label_happy', 'Label_neutral', 'Label_sad', 'Label_surprise']]

텍스트 시퀀스를 정수 시퀀스로 변환

In [7]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X_data_txt)
tokenizer.word_index
print(tokenizer.word_index)

{"'어'": 1, "'아'": 2, "'거'": 3, "'야'": 4, "'뭐'": 5, "'있'": 6, "'나'": 7, "'해'": 8, "'아니'": 9, "'내'": 10, "'지금'": 11, "'그'": 12, "'씨'": 13, "'왜'": 14, "'겠'": 15, "'안'": 16, "'되'": 17, "'면'": 18, "'사람'": 19, "'말'": 20, "'오빠'": 21, "'좀'": 22, "'저'": 23, "'만'": 24, "'알'": 25, "'없'": 26, "'보'": 27, "'우리'": 28, "'진짜'": 29, "'는데'": 30, "'었'": 31, "'봐'": 32, "'근데'": 33, "'했'": 34, "'습니다'": 35, "'이거'": 36, "'같'": 37, "'죠'": 38, "'았'": 39, "'아이'": 40, "'일'": 41, "'수'": 42, "'좋'": 43, "'너'": 44, "'자'": 45, "'그럼'": 46, "'잖아'": 47, "'어요'": 48, "'시'": 49, "'님'": 50, "'할'": 51, "'그냥'": 52, "'어떡'": 53, "'됐'": 54, "'살'": 55, "'제'": 56, "'잘'": 57, "'랑'": 58, "'요'": 59, "'으로'": 60, "'응'": 61, "'못'": 62, "'당신'": 63, "'그게'": 64, "'에서'": 65, "'무슨'": 66, "'니까'": 67, "'죽'": 68, "'그래'": 69, "'생각'": 70, "'서'": 71, "'그런'": 72, "'사랑'": 73, "'주'": 74, "'엄마'": 75, "'이게'": 76, "'개'": 77, "'얘기'": 78, "'이렇게'": 79, "'먹'": 80, "'음'": 81, "'한테'": 82, "'맞'": 83, "'싶'": 84, "'적'": 85, "'라고'": 86, "'하나'": 87, "'아이디어'": 88, "'

In [8]:
vocab_size = len(tokenizer.word_index)
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_data_txt)
X_data_txt = tokenizer.texts_to_sequences(X_data_txt)
print(X_data_txt)

[[155, 1158, 28, 113, 172, 4, 136], [1401, 1867, 139, 2264, 2265, 4, 38, 235, 6, 2266, 2267, 2268, 23, 1868, 987, 162, 102, 35, 39], [10], [427, 177, 427, 17, 1645, 160, 356, 49], [30, 21, 21, 673, 1869, 1870, 60, 1870, 1401, 2269, 2270, 73, 121, 1113, 114, 72], [114, 72, 6, 428, 699, 338, 2271, 1505, 112, 97, 16, 31], [3, 114, 137, 112, 97, 16, 120, 1871, 427, 7, 32, 289], [1871, 66, 93, 58, 218, 50, 226, 15, 1024, 137, 752, 15, 1872, 61, 2272, 137, 752, 1114, 158, 11, 940, 345, 64, 93, 21, 52, 119, 10, 87, 28, 31], [12, 64, 21, 395, 139], [1025, 1334, 12, 1646, 2273, 50, 31], [396, 1402, 21, 58, 137, 152, 1402, 1647, 10], [542, 2274, 50, 39, 542, 1115, 700, 1158, 28, 50, 39], [162, 5, 7, 16, 39, 155, 81, 2, 28, 4, 2275, 58, 2276, 152], [57, 149, 25, 88, 166, 2, 28, 16, 36, 941, 81, 105, 125, 1401, 1867, 100, 35, 31], [1648, 1402, 1506, 110, 2277, 377, 1649, 1401, 44, 3, 533, 1648, 20, 7, 284], [47, 88, 25, 102, 2278, 427, 1869, 602, 7, 73, 427, 451, 1282], [57, 2279, 1650, 80, 166, 2

패딩 작업

In [9]:
max_len = max(len(text) for text in X_data_txt)
X_data_txt = pad_sequences(X_data_txt, maxlen=max_len)

X데이터와 Y데이터를 각각 train, test로 분리

In [10]:
X_train_txt, X_test_txt=train_test_split(X_data_txt, test_size = 0.2, shuffle=False)
Y_train_txt, Y_test_txt=train_test_split(Y_data_txt, test_size = 0.2, shuffle=False)

text 모델 load

In [11]:
model_path='/content/gdrive/MyDrive/Human_understand/KEMDy19/best_model_txt.h5'

In [12]:
loaded_text_model = load_model(model_path)

Predict하기

In [13]:
predict_txt = loaded_text_model.predict(X_test_txt)
predict_txt_accuracy=loaded_text_model.evaluate(X_test_txt, Y_test_txt)
print(predict_txt)
print(predict_txt_accuracy[1])

58/58 [==============================] - 1s 5ms/step - loss: 0.6023 - accuracy: 0.7791
[[6.2229177e-05 6.5831315e-05 2.0516347e-05 ... 4.1302536e-03
  7.2774623e-04 1.6121683e-04]
 [3.3724948e-04 1.2544834e-04 8.1629551e-05 ... 2.1135136e-03
  1.8321698e-03 2.5427615e-04]
 [3.1119171e-03 8.9226807e-05 4.4729179e-05 ... 5.1355183e-02
  6.4382744e-05 1.7009167e-02]
 ...
 [7.7155727e-01 9.4465213e-03 2.0716158e-01 ... 1.1867228e-04
  1.1990551e-03 1.0381555e-02]
 [8.5979933e-01 1.6094077e-02 1.0785851e-01 ... 1.5279483e-03
  6.6096587e-03 6.0729613e-03]
 [3.0993727e-01 1.3668634e-01 9.0970941e-02 ... 3.2393318e-02
  4.1128555e-01 1.3884776e-02]]
0.7791142463684082


In [14]:
# 예측한 one-hot encoding의 인덱스가 출력됨
predict_classes_txt = predict_txt.argmax(axis=1)
print(predict_classes_txt[:20])
print(predict_classes_txt.shape)

Y_test_txt_values = Y_test_txt.values
Y_test_classes_txt = np.argmax(Y_test_txt_values, axis=1)

print(Y_test_classes_txt[:20])
print(Y_test_classes_txt.shape)

[3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 0]
(1829,)
[3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 3]
(1829,)


In [15]:
isCorrect_txt = list(predict_classes_txt == Y_test_classes_txt)
isCorrect_txt_True = isCorrect_txt.count(True)
print('Text 모델이 맞춘 갯수',isCorrect_txt_True)
isCorrect_txt_False = isCorrect_txt.count(False)
print('Text 모델이 틀린 갯수', isCorrect_txt_False)

Text 모델이 맞춘 갯수 1425
Text 모델이 틀린 갯수 404


# Text F1 score

In [17]:
# confusion matrix & 정밀도 & 재현율 구하기
get_model_metrics(Y_test_classes_txt, predict_classes_txt)

Confusion Matrix
[[391   6   3   4  46  11  10]
 [ 45  36   3   2  44   2   3]
 [  8   0  32   3   2   0   9]
 [  7   0   1 238  69   0  13]
 [ 46   1   1  21 599  17  15]
 [  2   1   0   1   7  89   0]
 [  0   0   0   0   1   0  40]]
정밀도:0.794, 재현율:0.779, f1_score:0.771


##Bio 모델

데이터셋 불러오기

In [18]:
merged_dataset=pd.read_csv("/content/gdrive/MyDrive/Human_understand/KEMDy19/merged_bio_text.csv")

데이터셋을 train데이터와 test데이터로 분리

In [19]:
train_data_bio, test_data_bio=train_test_split(merged_dataset, test_size=0.2, shuffle=False)
print(len(train_data_bio), len(test_data_bio))#데이터량 

7316 1829


train_data와 test_data를 각각 x_train,y_train와 x_test,y_test로 분리

In [20]:
X_train_bio = train_data_bio[['mean_EDA', 'mean_ECG', 'mean_Temp']]
Y_train_bio = train_data_bio[['Label_angry', 'Label_disgust', 'Label_fear', 'Label_happy', 'Label_neutral', 'Label_sad', 'Label_surprise']]
X_test_bio = test_data_bio[['mean_EDA', 'mean_ECG', 'mean_Temp']]
Y_test_bio = test_data_bio[['Label_angry', 'Label_disgust', 'Label_fear', 'Label_happy', 'Label_neutral', 'Label_sad', 'Label_surprise']]

bio 모델 load

In [21]:
loaded_bio_model = load_model('/content/gdrive/MyDrive/Human_understand/KEMDy19/best_model_bio.h5')

Predict 하기

In [22]:
predict_bio = loaded_bio_model.predict(X_test_bio)
predict_bio_accuracy=loaded_bio_model.evaluate(X_test_bio, Y_test_bio)
print(predict_bio)
print(predict_bio_accuracy[1])

58/58 [==============================] - 0s 2ms/step - loss: 1.1657 - accuracy: 0.5560
[[3.2597072e-02 2.1123389e-02 3.4166724e-04 ... 4.8484954e-01
  1.1195687e-03 3.3948960e-04]
 [2.3261743e-02 1.5771125e-02 1.4342622e-04 ... 5.0170052e-01
  1.0156078e-03 8.1609025e-05]
 [3.4004167e-02 1.6849810e-02 4.8130046e-04 ... 5.6314743e-01
  3.4187674e-03 3.9461761e-04]
 ...
 [2.1645167e-01 3.6781626e-03 3.5649635e-02 ... 3.1552231e-03
  2.6569156e-08 7.4068809e-01]
 [7.3597413e-01 1.3195874e-02 8.8242956e-02 ... 2.0214155e-02
  5.0147751e-08 1.4066611e-01]
 [7.3731136e-01 1.5260884e-02 8.8664375e-02 ... 2.7914345e-02
  7.9458246e-08 1.2819400e-01]]
0.5560415387153625


In [23]:
# 예측한 one-hot encoding의 인덱스가 출력됨
predict_classes_bio = predict_bio.argmax(axis=1)
print(predict_classes_bio[:20])
print(predict_classes_bio.shape)

Y_test_bio_values = Y_test_bio.values
Y_test_classes_bio = np.argmax(Y_test_bio_values, axis=1)

print(Y_test_classes_bio[:20])
print(Y_test_classes_bio.shape)

[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 0 0 0 0 0 0 0]
(1829,)
[3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 3 3 3 3 3 3 3 3 3 3 4 4 3 3 3 3 3 3
 4 4 4 4 4 4 4 0 0 0 0 0 0]
(1829,)


In [24]:
isCorrect_bio = list(predict_classes_bio == Y_test_classes_bio)
isCorrect_bio_True = isCorrect_bio.count(True)
print('bio 모델이 맞춘 갯수',isCorrect_bio_True)
isCorrect_bio_False = isCorrect_bio.count(False)
print('bio 모델이 틀린 갯수', isCorrect_bio_False)

bio 모델이 맞춘 갯수 1017
bio 모델이 틀린 갯수 812


# Bio F1 score

In [26]:
# confusion matrix & 정밀도 & 재현율 구하기
get_model_metrics(Y_test_classes_bio, predict_classes_bio)

Confusion Matrix
[[188   0   0  14 149   7 113]
 [ 28   0   0   7  76   0  24]
 [ 13   0   0   1   0   0  40]
 [  0   0   0 171 114   3  40]
 [ 24   0   0  62 571  10  33]
 [  0   0   0   1  53  46   0]
 [  0   0   0   0   0   0  41]]
정밀도:0.579, 재현율:0.556, f1_score:0.537


##wav 모델

데이터셋 불러오기

In [27]:
X_data_wav = '/content/gdrive/MyDrive/Human_understand/KEMDy19/needed_mfcc.npy'
Y_data_wav = '/content/gdrive/MyDrive/Human_understand/KEMDy19/needed_mfcc_y.pkl'

피클 파일 및 넘파이 파일 로드하기

In [28]:
#MFCC파일(X)
X_data_wav = np.load(file = X_data_wav)

#Label 파일(Y)
with open(Y_data_wav,"rb") as fr:
    Y_data_wav_list = pickle.load(fr)

Y데이터 one hot encoding

In [29]:
#리스트 형식으로 Label을 불러오기 때문에 리스트를 데이터 프레임으로 변환
Y_data_wav=pd.DataFrame(Y_data_wav_list, columns=["Label"])

#one hot e/content/gdrive/MyDrive/Human_understand/KEMDy19/needed_mfcc.npyncoding
Y_data_wav = pd.get_dummies(Y_data_wav, columns = ['Label'])
Y_data_wav # one hot encoding 확인

,Label_angry,Label_disgust,Label_fear,Label_happy,Label_neutral,Label_sad,Label_surprise
0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
9140,0,0,0,0,0,0,1
9141,0,0,0,0,0,0,1
9142,0,0,0,0,0,0,1
9143,0,0,1,0,0,0,0


X데이터,Y데이터를 각각 train과 test로 분리

In [30]:
X_train_wav, X_test_wav=train_test_split(X_data_wav, test_size=0.2, shuffle=False)
Y_train_wav, Y_test_wav=train_test_split(Y_data_wav, test_size=0.2, shuffle=False)
#데이터량 확인
print(len(X_train_wav), len(Y_train_wav))
print(len(X_test_wav), len(Y_test_wav))

7316 7316
1829 1829


wav 모델 load

In [31]:
model_path='/content/gdrive/MyDrive/Human_understand/KEMDy19/best_model_wav.h5'

In [32]:
loaded_wav_model = load_model(model_path)

Predict하기

In [33]:
predict_wav = loaded_wav_model.predict(X_test_wav)
predict_wav_accuracy=loaded_wav_model.evaluate(X_test_wav, Y_test_wav)
print(predict_wav)
print(predict_wav_accuracy[1])

58/58 [==============================] - 1s 7ms/step - loss: 1.0025 - accuracy: 0.7376
[[3.8704397e-06 5.5974242e-06 1.0658864e-08 ... 1.1584331e-04
  8.1194720e-08 3.8344602e-08]
 [1.3587800e-02 9.6814460e-01 1.4530168e-03 ... 4.8608994e-03
  2.4423916e-03 9.6589478e-04]
 [1.5288849e-04 5.4722500e-06 3.1529883e-08 ... 6.8525749e-04
  2.9790135e-07 1.4481168e-06]
 ...
 [6.7912480e-03 1.0907373e-06 3.9886445e-04 ... 1.0408490e-03
  1.9296209e-03 9.8486567e-01]
 [8.7062926e-06 2.6832644e-05 9.9728274e-01 ... 2.0531083e-05
  2.6289676e-03 1.9815872e-05]
 [3.3099508e-01 3.2516736e-01 3.0178526e-02 ... 1.1915720e-01
  1.1578434e-01 6.5243199e-02]]
0.7375615239143372


In [34]:
# 예측한 one-hot encoding의 인덱스가 출력됨
predict_classes_wav = predict_wav.argmax(axis=1)
print(predict_classes_wav[:20])
print(predict_classes_wav.shape)

Y_test_wav_values = Y_test_wav.values
Y_test_classes_wav = np.argmax(Y_test_wav_values, axis=1)

print(Y_test_classes_wav[:20])
print(Y_test_classes_wav.shape)

[3 1 3 4 4 4 3 4 4 4 4 4 4 4 4 4 5 3 3 3]
(1829,)
[3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 3]
(1829,)


In [35]:
isCorrect_wav = list(predict_classes_wav == Y_test_classes_wav)
isCorrect_wav_True = isCorrect_wav.count(True)
print('wav 모델이 맞춘 갯수',isCorrect_wav_True)
isCorrect_wav_False = isCorrect_wav.count(False)
print('wav 모델이 틀린 갯수', isCorrect_wav_False)

wav 모델이 맞춘 갯수 1349
wav 모델이 틀린 갯수 480


# Wav F1 score

In [37]:
# confusion matrix & 정밀도 & 재현율 구하기
get_model_metrics(Y_test_classes_wav, predict_classes_wav)

Confusion Matrix
[[153  22   1   2  19   3   1]
 [  1  83   0   1   1   1   0]
 [  2  10  87   2  15   8   3]
 [ 32  30   3 157  45   5   0]
 [ 41  85  14  15 676  14  16]
 [  9   9   2   2  15  83   3]
 [ 13  14   1   3   9   8 110]]
정밀도:0.792, 재현율:0.738, f1_score:0.751


#Ensemble 모델

In [38]:
#하나의 모델 predict를 데이터 프레임으로 만들기
df_bio_predict = pd.DataFrame(predict_bio)
df_txt_predict = pd.DataFrame(predict_txt)
df_wav_predict = pd.DataFrame(predict_wav)

#같은 위치로 더한 데이터프레임 생성
plus_predict = pd.DataFrame()
plus_predict = df_bio_predict+df_txt_predict+df_wav_predict

#데이터 프레임에서 하나의 행 중에 최대값이 어느 인덱스에 있는지 예상 정답값을 리스트에 저장
predict=[]
for i in range(len(plus_predict)):
  max_index= plus_predict.loc[i,:].idxmax(axis=0)
  predict.append(max_index)

#Y_test에서 1이 들어가는 있는 부분을 인덱스로 리스트에 저장
Y_test_index=[]
Y_test_bio= np.array(Y_test_bio.values)
for row in Y_test_bio:
  index=row.argmax(axis=0)
  Y_test_index.append(index)

# #예상 값 출력
# for label in predict:
#   if label==0:
#     print('angry')
#   elif label==1:
#     print('disgust')
#   elif label==2:
#     print('fear')
#   elif label==3:
#     print('happy')
#   elif label==4:
#     print('neutral')
#   elif label==5:
#     print('sad')
#   elif label==6:
#     print('surprise')

print('Y_test_index', Y_test_index)
print('Y_test_predict', predict)
#Y_test_index의 정답 인덱스와 answer에 있는 예측 인덱스가 맞으면 count 1 증가 시키기
dataset_size=len(predict)
count=0
for i in range(dataset_size):
  if predict[i]==Y_test_index[i]:
    count+=1
print(count)

#accuracy  계산
print("accuracy: ", count/dataset_size)

Y_test_index [3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 4, 1, 1, 1, 0, 4, 4, 1, 1, 3, 3, 6, 6, 3, 3, 3, 3, 3, 4, 1, 4, 3, 3, 3, 4, 1, 0, 1, 0, 0, 4, 1, 1, 3, 3, 3, 3, 6, 6, 3, 3, 0, 3, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 4, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 4, 0, 5, 0, 4, 5, 5, 5, 5, 5, 5, 0, 5, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 1, 4, 4, 4, 4, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 1, 1, 4, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 1, 4, 1, 1, 4, 0, 0, 6, 0, 0, 2, 2, 2, 1, 4, 1, 2, 1, 0, 0, 0, 0, 0, 4, 6, 6, 6, 0, 0, 6, 0, 0, 4, 2, 0, 6, 2, 2, 2, 2, 0, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 6, 6, 6, 0, 0, 1, 4, 4, 4, 4, 4, 0, 3, 3, 3, 4, 1, 3, 3, 4, 0, 0, 0, 4, 4, 4, 4, 3, 3, 3,

# Ensemble F1 score

In [39]:
# confusion matrix & 정밀도 & 재현율 구하기
get_model_metrics(predict, Y_test_index)

Confusion Matrix
[[335  10   4   2   9   2   0]
 [ 11  69   2   2   3   1   0]
 [ 12   1  25   1   0   0   0]
 [  4   0   2 247   4   0   0]
 [ 77  50   8  60 665   6   0]
 [  9   0   0   0  11  91   0]
 [ 23   5  13  16   8   0  41]]
정밀도:0.844, 재현율:0.805, f1_score:0.806
